In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
#from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
#from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K
import gc
import time

D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\toolkit

In [2]:
#help(keras.applications.mobilenetv2)
help(keras.applications.mobilenetv2)

Help on module keras.applications.mobilenetv2 in keras.applications:

NAME
    keras.applications.mobilenetv2

DATA
    absolute_import = _Feature((2, 5, 0, 'alpha', 1), (3, 0, 0, 'alpha', 0...
    division = _Feature((2, 2, 0, 'alpha', 2), (3, 0, 0, 'alpha', 0), 8192...
    print_function = _Feature((2, 6, 0, 'alpha', 2), (3, 0, 0, 'alpha', 0)...

FILE
    d:\toolkits.win\anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\keras\applications\mobilenetv2.py




In [3]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [4]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [5]:
def DataSet(iFold,noise_tag,dup_tag,rand_tag):
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    side_len  = math.ceil(math.sqrt(num_feature))
    num_class = 2
    #
    mat_ind_path =("../Data_CNN_Indus/Mat_ind_F%d.txt") % (iFold)
    #
    mat_ind = file2array(mat_ind_path, "\t")
    
    if (int)(rand_tag):
        tmp_vec = np.zeros(num_feature)
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            tmp_vec[index] = int(mat_ind[r,c])
        index = [i for i in range(num_feature)]
        random.shuffle(index)
        tmp_vec = tmp_vec[index]
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            mat_ind[r,c] = int(tmp_vec[index])

    tmp_cnt_per_class = np.zeros(num_class)
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_cnt_per_class[(int)(tmp)] = tmp_cnt_per_class[(int)(tmp)] + 1
        
    tmp_max = tmp_cnt_per_class[0]
    tmp_ind = 0
    for index in range(num_class):
        if tmp_cnt_per_class[index] > tmp_max:
            tmp_max = tmp_cnt_per_class[index]
            tmp_ind = index
            
    num_dup = np.zeros(num_class)
    for index in range(num_class):
        if (int)(dup_tag):
            num_dup[index] = math.floor(tmp_cnt_per_class[tmp_ind]/tmp_cnt_per_class[index])
        else:
            num_dup[index] = 1
        
    nrow_train_dup = 0
    for index in range(num_class):
        nrow_train_dup = nrow_train_dup + tmp_cnt_per_class[index] * num_dup[index]
        
    nrow_train_dup = (int)(nrow_train_dup)
    
    X_train = np.empty((nrow_train_dup, side_len, side_len, 1), dtype=float)
    Y_train = np.empty((nrow_train_dup, num_class), dtype=float)

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_cnt = 0
            for i in range(side_len):
                for j in range(side_len):
                    if tmp_cnt < num_feature:
                        tmp_ind = mat_ind[i,j]
                        X_train[sam_count,i,j,0] = float(data_train[index,(int)(tmp_ind)])
                    else:
                        X_train[sam_count,i,j,0] = 0.5
                    tmp_cnt = tmp_cnt + 1
            sam_count = sam_count + 1

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_v = np.zeros(num_class)
            tmp_v[(int)(tmp)] = 1
            Y_train[sam_count] = tmp_v
            sam_count = sam_count + 1
    
    X_test  = np.empty((nrow_test, side_len, side_len, 1), dtype=float)
    Y_test  = np.empty((nrow_test, num_class), dtype=float)

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(side_len):
            for j in range(side_len):
                if tmp_cnt < num_feature:
                    tmp_ind = mat_ind[i,j]
                    tmp_var = float(data_test[index,(int)(tmp_ind)])
                    #print(tmp_var)
                    #if tmp_var < 0:
                    #    tmp_var = 0
                    #if tmp_var > 1:
                    #    tmp_var = 1
                    X_test[index,i,j,0] = tmp_var
                else:
                    X_test[index,i,j,0] = 0.5
                tmp_cnt = tmp_cnt + 1
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_test[index] = tmp_v
        
    if (int)(noise_tag):
        noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
        output = X_train + noise
        X_train = np.concatenate((X_train, output), axis=0)
        Y_train = np.concatenate((Y_train, Y_train), axis=0)
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
#    X_train_new = np.empty((len(X_train), 224, 224, 3))
#    #Y_train_new = Y_train
#    
#    for index in range(len(X_train)):
#        tmp_new = ndarray_nearest_neighbour_scaling(X_train[index],224,224)
#        for c in range(3):
#            X_train_new[index,:,:,c] = tmp_new[:,:,0]
#    
#    X_test_new = np.empty((len(X_test), 224, 224, 3))
#    #Y_test_new = Y_test
#    
#    for index in range(len(X_test)):
#        tmp_new = ndarray_nearest_neighbour_scaling(X_test[index],224,224)
#        for c in range(3):
#            X_test_new[index,:,:,c] = tmp_new[:,:,0]
#            
#    del X_train, X_test
#    gc.collect()

    return X_train,Y_train,X_test,Y_test

In [6]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [7]:
def neural(channel,height,width,classes):
    input_shape = (channel,height,width)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (height,width,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    model.add(Flatten())
    #model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(classes,activation="softmax",name="fc2"))
    return model


In [8]:
def get_statistics(model,sam_data,lab_data):
    num_class = 2

    #
    test_rows = len(lab_data)
    test_prc = np.zeros(num_class)
    test_rec = np.zeros(num_class)
    test_Fva = np.zeros(num_class)
    test_BER = np.zeros(num_class)
    tmp_beta = 1
    yhat_out    = model.predict(sam_data, batch_size=32, verbose=1)
    yhat_label  = np.zeros(len(lab_data))
    true_labels = np.zeros(len(lab_data))
    for index in range(len(lab_data)):
        tmp_vec = lab_data[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind
        #
        tmp_vec = yhat_out[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        yhat_label[index] = tmp_ind

    test_N_TP = np.zeros(num_class)
    test_N_FP = np.zeros(num_class)
    test_N_TN = np.zeros(num_class)
    test_N_FN = np.zeros(num_class)
    tmp_count = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    test_N_TP[j]=test_N_TP[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_FP[j]=test_N_FP[j]+1
            else:
                if j == true_label:
                    test_N_FN[j]=test_N_FN[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_TN[j]=test_N_TN[j]+1

    for i in range(num_class):
        if test_N_TP[i] > 0:
            test_prc[i] = test_N_TP[i] / (test_N_TP[i] + test_N_FP[i])
        else:
            test_prc[i] = 0

        test_rec[i]= test_N_TP[i] / (test_N_TP[i] + test_N_FN[i])
        test_Fva[i]=(1 + tmp_beta*tmp_beta)*test_rec[i]*test_prc[i]/\
                      (tmp_beta*tmp_beta*(test_rec[i]+test_prc[i]))
        test_BER[i]= test_N_FN[i] / (test_N_TP[i] + test_N_FN[i])
        
    print(tmp_count)
        
    return test_N_TP,test_N_FP,test_N_TN,test_N_FN,tmp_count
    

In [9]:
all_TP_train=[]
all_FP_train=[]
all_TN_train=[]
all_FN_train=[]
all_count_train=[]
all_TP_test=[]
all_FP_test=[]
all_TN_test=[]
all_FN_test=[]
all_count_test=[]
all_time_train=[]
all_time_predict_train=[]
all_time_predict_test=[]

nRun = 10
side_len = 22
num_class = 2
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for fold in range(nRun):
                iFold = fold + 1
                X_train,Y_train,X_test,Y_test = DataSet(iFold,noise_tag,dup_tag,ran_tag)
                print('X_train shape : ',X_train.shape)
                print('Y_train shape : ',Y_train.shape)
                print('X_test shape : ',X_test.shape)
                print('Y_test shape : ',Y_test.shape)
                #print(X_train[0])
                #print(Y_train[0,1])
                print(noise_tag,dup_tag,ran_tag,iFold)

                # # model
                #model = tf.keras.applications.mobilenet.MobileNet(
                #    input_shape=None, 
                #    alpha=1.0, 
                #    depth_multiplier=1, 
                #    include_top=True, 
                #    weights=None, 
                #    input_tensor=None, 
                #    pooling='max', 
                #    classes=2
                #)

                ##
                model = neural(1,side_len,side_len,num_class)

                model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

                # # train
                start = time.clock()
                model.fit(X_train, Y_train, epochs=10, batch_size=16)
                all_time_train.append(time.clock()-start)

                # # evaluate
                #model.evaluate(X_test, Y_test, batch_size=32)

                # #
                #models.append(model)

                # # save
                #save_path =("./my_resnet_model.h5_F%d") % (iFold)
                #model.save(save_path)

                #
                X_train,Y_train,X_test,Y_test = DataSet(iFold,0,0,ran_tag)
                start = time.clock()
                train_N_TP,train_N_FP,train_N_TN,train_N_FN,train_count = get_statistics(model,X_train,Y_train)
                all_time_predict_train.append(time.clock()-start)
                start = time.clock()
                test_N_TP,test_N_FP,test_N_TN,test_N_FN,test_count = get_statistics(model,X_test,Y_test)
                all_time_predict_test.append(time.clock()-start)
                all_TP_train.append(train_N_TP)
                all_FP_train.append(train_N_FP)
                all_TN_train.append(train_N_TN)
                all_FN_train.append(train_N_FN)
                all_count_train.append(train_count)
                all_TP_test.append(test_N_TP)
                all_FP_test.append(test_N_FP)
                all_TN_test.append(test_N_TN)
                all_FN_test.append(test_N_FN)
                all_count_test.append(test_count)


X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
0 0 0 1
Epoch 1/10
1410/1410 [==============================] - 4s 3ms/step - loss: 0.3115 - acc: 0.8993
Epoch 2/10
1410/1410 [==============================] - 1s 447us/step - loss: 0.2496 - acc: 0.9333
Epoch 3/10
1410/1410 [==============================] - 1s 447us/step - loss: 0.2480 - acc: 0.9333
Epoch 4/10
1410/1410 [==============================] - 1s 433us/step - loss: 0.2461 - acc: 0.9333
Epoch 5/10
1410/1410 [==============================] - 1s 440us/step - loss: 0.2465 - acc: 0.9333
Epoch 6/10
1410/1410 [==============================] - 1s 454us/step - loss: 0.2453 - acc: 0.9333
Epoch 7/10
1410/1410 [==============================] - 1s 496us/step - loss: 0.2472 - acc: 0.9333
Epoch 8/10
1410/1410 [==============================] - 1s 468us/step - loss: 0.2434 - acc: 0.9333
Epoch 9/10
1410/1410 [==============================] - 1s 454us/step - loss: 0.24

D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars


X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
0 0 0 2
Epoch 1/10
1410/1410 [==============================] - 1s 783us/step - loss: 0.2664 - acc: 0.9333
Epoch 2/10
1410/1410 [==============================] - 1s 539us/step - loss: 0.2480 - acc: 0.9333
Epoch 3/10
1410/1410 [==============================] - 1s 539us/step - loss: 0.2449 - acc: 0.9333
Epoch 4/10
1410/1410 [==============================] - 1s 532us/step - loss: 0.2453 - acc: 0.9333
Epoch 5/10
1410/1410 [==============================] - 1s 539us/step - loss: 0.2490 - acc: 0.9333
Epoch 6/10
1410/1410 [==============================] - 1s 539us/step - loss: 0.2426 - acc: 0.9333
Epoch 7/10
1410/1410 [==============================] - 1s 539us/step - loss: 0.2404 - acc: 0.9333
Epoch 8/10
1410/1410 [==============================] - 1s 468us/step - loss: 0.2376 - acc: 0.9333
Epoch 9/10
1410/1410 [==============================] - 1s 475us/step - loss: 0.

1411/1411 [==============================] - 1s 468us/step - loss: 0.2490 - acc: 0.9334
Epoch 5/10
1411/1411 [==============================] - 1s 468us/step - loss: 0.2459 - acc: 0.9334
Epoch 6/10
1411/1411 [==============================] - 1s 468us/step - loss: 0.2406 - acc: 0.9334
Epoch 7/10
1411/1411 [==============================] - 1s 454us/step - loss: 0.2449 - acc: 0.9334
Epoch 8/10
1411/1411 [==============================] - 1s 454us/step - loss: 0.2419 - acc: 0.9334
Epoch 9/10
1411/1411 [==============================] - 1s 439us/step - loss: 0.2393 - acc: 0.9334
Epoch 10/10
1411/1411 [==============================] - 0s 227us/step
[1317.   94.]
156/156 [==============================] - 0s 64us/step
[146.  10.]
X_train shape :  (1411, 22, 22, 1)
Y_train shape :  (1411, 2)
X_test shape :  (156, 22, 22, 1)
Y_test shape :  (156, 2)
0 0 0 9
Epoch 1/10
1411/1411 [==============================] - 1s 1ms/step - loss: 0.3201 - acc: 0.8958
Epoch 2/10
1411/1411 [=================

1410/1410 [==============================] - 0s 268us/step
[1317.   93.]
157/157 [==============================] - 0s 115us/step
[146.  11.]
X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
0 0 1 5
Epoch 1/10
1410/1410 [==============================] - 2s 1ms/step - loss: 0.3125 - acc: 0.9113
Epoch 2/10
1410/1410 [==============================] - 1s 730us/step - loss: 0.2455 - acc: 0.9340
Epoch 3/10
1410/1410 [==============================] - 1s 596us/step - loss: 0.2448 - acc: 0.9340
Epoch 4/10
1410/1410 [==============================] - 1s 482us/step - loss: 0.2456 - acc: 0.9340
Epoch 5/10
1410/1410 [==============================] - 1s 482us/step - loss: 0.2462 - acc: 0.9340
Epoch 6/10
1410/1410 [==============================] - 1s 496us/step - loss: 0.2446 - acc: 0.9340
Epoch 7/10
1410/1410 [==============================] - 1s 461us/step - loss: 0.2423 - acc: 0.9340
Epoch 8/10
1410/1410 [=================

...

2820/2820 [==============================] - 1s 514us/step - loss: 0.2326 - acc: 0.9333
Epoch 8/10
2820/2820 [==============================] - 1s 493us/step - loss: 0.2286 - acc: 0.9333
Epoch 9/10
2820/2820 [==============================] - 1s 521us/step - loss: 0.2240 - acc: 0.9333
Epoch 10/10
1410/1410 [==============================] - 1s 855us/step
[1316.   94.]
157/157 [==============================] - 0s 127us/step
[147.  10.]
X_train shape :  (2820, 22, 22, 1)
Y_train shape :  (2820, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
1 0 1 4
Epoch 1/10
2820/2820 [==============================] - 4s 2ms/step - loss: 0.2594 - acc: 0.9340
Epoch 2/10
2820/2820 [==============================] - 1s 532us/step - loss: 0.2439 - acc: 0.9340
Epoch 3/10
2820/2820 [==============================] - 1s 514us/step - loss: 0.2390 - acc: 0.9340
Epoch 4/10
2820/2820 [==============================] - 1s 518us/step - loss: 0.2364 - acc: 0.9340
Epoch 5/10
2820/2820 [================

Epoch 1/10
2822/2822 [==============================] - 4s 2ms/step - loss: 0.2913 - acc: 0.9150
Epoch 2/10
2822/2822 [==============================] - 1s 482us/step - loss: 0.2484 - acc: 0.9334
Epoch 3/10
2822/2822 [==============================] - 1s 489us/step - loss: 0.2468 - acc: 0.9334
Epoch 4/10
2822/2822 [==============================] - 1s 482us/step - loss: 0.2445 - acc: 0.9334
Epoch 5/10
2822/2822 [==============================] - 1s 503us/step - loss: 0.2421 - acc: 0.9334
Epoch 6/10
2822/2822 [==============================] - 1s 482us/step - loss: 0.2377 - acc: 0.9334
Epoch 7/10
2822/2822 [==============================] - 1s 485us/step - loss: 0.2341 - acc: 0.9334
Epoch 8/10
2822/2822 [==============================] - 1s 514us/step - loss: 0.2282 - acc: 0.9334
Epoch 9/10
2822/2822 [==============================] - 1s 503us/step - loss: 0.2215 - acc: 0.9334
Epoch 10/10
1411/1411 [==============================] - 1s 945us/step
[1317.   94.]
156/156 [=================

5238/5238 [==============================] - 3s 500us/step - loss: 0.2909 - acc: 0.9141
Epoch 8/10
5238/5238 [==============================] - 3s 525us/step - loss: 0.2480 - acc: 0.9351
Epoch 9/10
5238/5238 [==============================] - 3s 523us/step - loss: 0.2154 - acc: 0.9441
Epoch 10/10
1410/1410 [==============================] - 2s 1ms/step
[1317.   93.]
157/157 [==============================] - 0s 127us/step
[146.  11.]
X_train shape :  (5238, 22, 22, 1)
Y_train shape :  (5238, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
1 1 0 7
Epoch 1/10
5238/5238 [==============================] - 8s 1ms/step - loss: 0.6743 - acc: 0.5943A: 3s - 
Epoch 2/10
5238/5238 [==============================] - 3s 626us/step - loss: 0.5954 - acc: 0.7297
Epoch 3/10
5238/5238 [==============================] - 3s 565us/step - loss: 0.4928 - acc: 0.7973
Epoch 4/10
5238/5238 [==============================] - 3s 561us/step - loss: 0.4084 - acc: 0.8396
Epoch 5/10
5238/5238 [==========

Epoch 1/10
5264/5264 [==============================] - 6s 1ms/step - loss: 0.6767 - acc: 0.6007
Epoch 2/10
5264/5264 [==============================] - 3s 526us/step - loss: 0.6083 - acc: 0.7356
Epoch 3/10
5264/5264 [==============================] - 3s 534us/step - loss: 0.5128 - acc: 0.7751
Epoch 4/10
5264/5264 [==============================] - 3s 549us/step - loss: 0.4305 - acc: 0.8381
Epoch 5/10
5264/5264 [==============================] - 3s 507us/step - loss: 0.3612 - acc: 0.8708
Epoch 6/10
5264/5264 [==============================] - 3s 528us/step - loss: 0.3042 - acc: 0.9008
Epoch 7/10
5264/5264 [==============================] - 3s 524us/step - loss: 0.2581 - acc: 0.9257
Epoch 8/10
5264/5264 [==============================] - 3s 538us/step - loss: 0.2165 - acc: 0.9474
Epoch 9/10
5264/5264 [==============================] - 3s 540us/step - loss: 0.1857 - acc: 0.9571
Epoch 10/10
1410/1410 [==============================] - 2s 1ms/step
[1316.   94.]
157/157 [===================

5266/5266 [==============================] - 3s 499us/step - loss: 0.2955 - acc: 0.9132
Epoch 7/10
5266/5266 [==============================] - 3s 518us/step - loss: 0.2544 - acc: 0.9273
Epoch 8/10
5266/5266 [==============================] - 3s 505us/step - loss: 0.2193 - acc: 0.9447
Epoch 9/10
5266/5266 [==============================] - 3s 511us/step - loss: 0.1966 - acc: 0.9523
Epoch 10/10
1411/1411 [==============================] - 2s 1ms/step
[1317.   94.]
156/156 [==============================] - 0s 128us/step
[146.  10.]
X_train shape :  (5266, 22, 22, 1)
Y_train shape :  (5266, 2)
X_test shape :  (156, 22, 22, 1)
Y_test shape :  (156, 2)
1 1 1 10
Epoch 1/10
5266/5266 [==============================] - 8s 1ms/step - loss: 0.6699 - acc: 0.6206
Epoch 2/10
5266/5266 [==============================] - 3s 577us/step - loss: 0.6005 - acc: 0.7053
Epoch 3/10
5266/5266 [==============================] - 3s 558us/step - loss: 0.5191 - acc: 0.7653
Epoch 4/10
5266/5266 [=================

In [10]:
noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
output = X_train + noise
X_train_all = np.concatenate((X_train, output), axis=0)
print(noise)
print(noise.shape)
print(output.shape)
print(X_train.shape)
print(X_train_all.shape)
for i in range(len(X_train)):
    if (X_train[i]==X_train_all[i]).all()==False:
        print(i)
for i in range(len(X_train)):
    ind = len(X_train)+i
    if (output[i]==X_train_all[ind]).all()==False:
        print(ind)
        

[[[[ 8.50296269e-03]
   [-5.56475972e-03]
   [ 3.42121297e-03]
   ...
   [ 5.46674798e-03]
   [-1.02214239e-02]
   [-2.00023873e-02]]

  [[-7.52761359e-04]
   [ 9.19477509e-03]
   [-1.63164468e-02]
   ...
   [-2.23524707e-03]
   [ 1.09548838e-02]
   [ 1.31199959e-04]]

  [[ 1.50232130e-02]
   [-2.30541161e-02]
   [ 3.16930148e-03]
   ...
   [ 5.01518461e-03]
   [ 5.21135643e-03]
   [-4.61353043e-03]]

  ...

  [[ 2.33921760e-03]
   [-1.01453942e-02]
   [ 4.49912312e-03]
   ...
   [ 2.63150002e-03]
   [-2.62682877e-03]
   [ 2.56222265e-03]]

  [[ 1.72609447e-02]
   [-1.28579921e-02]
   [-4.47856851e-04]
   ...
   [ 1.15752750e-02]
   [ 1.70343152e-04]
   [ 1.27640484e-03]]

  [[-1.24443477e-02]
   [ 8.22129080e-04]
   [ 1.69245960e-02]
   ...
   [-2.53824988e-03]
   [ 1.72615659e-04]
   [ 5.94375837e-03]]]


 [[[-6.54495480e-03]
   [ 1.69320213e-03]
   [ 4.79727483e-03]
   ...
   [-4.29202848e-03]
   [ 7.48381851e-04]
   [ 3.71791749e-03]]

  [[-6.48818155e-03]
   [-4.62378244e-03]
   [

In [11]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_TP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
            for iRun in range(nRun):
                print(all_TP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])

for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_count_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_count_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[94.  0.]
[   0. 1317.]
[ 0. 94.]
[1317.    0.]
[94.  0.]
[   0. 1317.]
[ 0. 94.]
[1317.    0.]
[94.  0.]
[   0. 1317.]
[ 0. 94.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[10.  0.]
[  0. 146.]
[ 0. 10.]
[146.   0.]
[10.  0.]
[  0. 146.]
[ 0. 10.]
[146.   0.]
[10.  0.]
[  0. 146.]
[ 0. 10.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]
[   0. 1

[147.  10.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1316.   94.]
[147.  10.]


In [12]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


10.474079399999999
8.2052761
10.566729800000005
7.809778199999997
10.396565699999996
8.815853899999993
8.638515799999993
7.679264400000008
7.948457700000006
8.062190000000015
8.10998459999999
9.648600899999991
8.200213999999988
11.086993299999989
8.884777299999996
8.463542100000012
8.260268199999985
8.440377100000006
7.883828400000027
9.386100999999996
12.32399220000002
13.687968900000044
13.93952459999997
13.128839500000026
14.61753600000003
15.394177500000012
17.848960999999974
17.12773580000004
15.45100199999996
15.89456819999998
16.160633599999983
15.390416100000039
14.960268700000029
16.06409210000004
16.036344699999972
14.286610799999949
14.710119400000053
15.221066599999972
14.83031290000008
18.484388299999978
17.913834299999962
18.68053569999995
16.399488399999996
16.348285199999964
17.37220709999997
16.71924690000003
17.030851299999995
16.617836600000032
17.979089199999976
17.82861939999998
17.80940080000005
20.152065200000038
17.588625399999955
18.312359100000094
17.587956800

In [13]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


0.17117309999999897
0.2066941999999976
0.21088470000000115
0.22946980000000394
0.24923450000000003
0.2500102999999996
0.2849926000000096
0.3389364000000086
0.34690850000001205
0.32441950000000475
0.3422831999999971
0.3735704999999996
0.4055838000000165
0.40200459999999794
0.41176770000001284
0.4347191000000237
0.4801579000000231
0.500158200000044
0.4819056000000046
0.5449616000000219
0.5167668000000276
0.5559908000000178
0.5995896000000016
0.5785993999999732
0.6072867000000315
0.6245483000000149
0.6219646999999782
0.6647864999999911
0.7308663000000024
0.7473308000000998
0.7290946000000531
0.7440275999999812
0.7547974999999951
0.7973076999999194
0.8323246999999583
0.8476203999999825
0.8959370000000035
0.8770171999999548
0.8710062000000107
0.9093698000000359
1.0040122000000338
0.9845087999999578
0.9946327999999767
1.1027401000000054
1.0030160000000024
1.0131001999999398
1.0440882000000329
1.091043300000024
1.1373380999999654
1.163339599999972
1.16432779999991
1.1641916000000947
1.2164635

In [14]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


0.020113099999999662
0.020107800000001674
0.014558899999997266
0.014974199999997495
0.017266300000002843
0.016984899999997083
0.017021099999993794
0.019010100000002694
0.020063199999981407
0.01751280000001998
0.015451900000016394
0.0219372999999905
0.02075540000001297
0.020353799999980993
0.01664750000000481
0.018498800000003257
0.01822540000000572
0.01910689999999704
0.017213200000014695
0.016854799999975967
0.01698500000003378
0.027569400000004407
0.01709610000000339
0.017180199999984325
0.017811600000015915
0.01764709999997649
0.019620399999951132
0.01752729999998337
0.016471500000022843
0.018291999999973996
0.01684649999992871
0.016127999999980602
0.021414600000071005
0.016849500000034823
0.02425319999997555
0.018616699999938646
0.022824799999966672
0.02105230000006486
0.019251199999985147
0.017823499999963133
0.02042180000000826
0.024244600000088212
0.025909599999977218
0.02601320000007945
0.01920549999999821
0.018435900000099537
0.016879300000027797
0.01941260000000966
0.02081180

In [15]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))

#save_path =("./tmp_TP_EP%d") % (1)
#fl=open(save_path, 'w')
#lists=[line+"\n" for line in all_TP_train]
#fl.writelines(lists)